<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/query_engine/pandas_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas Query Engine

This guide shows you how to use our `PandasQueryEngine`: convert natural language to Pandas python code using LLMs.

The input to the `PandasQueryEngine` is a Pandas dataframe, and the output is a response. The LLM infers dataframe operations to perform in order to retrieve the result.

**NOTE**: We have measures in PandasQueryEngine to enforce safety and prevent arbitrary code execution. For instance, no execution of private/dunder methods, and access to a restricted set of globals.


If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index

In [1]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.query_engine import PandasQueryEngine


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Let's start on a Toy DataFrame

Here let's load a very simple dataframe containing city and population pairs, and run the `PandasQueryEngine` on it.

By setting `verbose=True` we can see the intermediate generated instructions.

In [2]:
# Test on some sample data
df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)

In [3]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [4]:
response = query_engine.query(
    "What is the city with the highest population?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['city'][df['population'].idxmax()]
```
> Pandas Output: Tokyo


In [5]:
display(Markdown(f"<b>{response}</b>"))

<b>Tokyo</b>

In [6]:
# get pandas python instructions
print(response.metadata["pandas_instruction_str"])

df['city'][df['population'].idxmax()]


## Analyzing the Titanic Dataset

The Titanic dataset is one of the most popular tabular datasets in introductory machine learning
Source: https://www.kaggle.com/c/titanic

#### Download Data

In [7]:
!wget 'https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/csv/titanic_train.csv' -O 'titanic_train.csv'

--2024-01-13 10:23:43--  https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/csv/titanic_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57726 (56K) [text/plain]
Saving to: ‘titanic_train.csv’

titanic_train.csv   100%[===================>]  56.37K  --.-KB/s    in 0.02s   

2024-01-13 10:23:44 (2.96 MB/s) - ‘titanic_train.csv’ saved [57726/57726]



In [8]:
df = pd.read_csv("./titanic_train.csv")

In [9]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [10]:
response = query_engine.query(
    "What is the correlation between survival and age?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['survived'].corr(df['age'])
```
> Pandas Output: -0.07722109457217755


In [11]:
display(Markdown(f"<b>{response}</b>"))

<b>-0.07722109457217755</b>

In [12]:
# get pandas python instructions
print(response.metadata["pandas_instruction_str"])

df['survived'].corr(df['age'])


## Additional Steps

### Analyzing / Modifying prompts

Let's look at the prompts! 

In [20]:
query_engine = PandasQueryEngine(df=df, verbose=True)
prompts = query_engine.get_prompts()
print(prompts["pandas_prompt"].template)

You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}
Query: {query_str}

Expression:


In [21]:
print(prompts["response_synthesis_prompt"].template)

KeyError: 'response_synthesis_prompt'

This is the instruction string (that you can customize by passing in `instruction_str` on initialization)

In [ ]:
instruction_str = """\
1. Convert the query to executable Python code using Pandas.
2. The final line of code should be a Python expression that can be called with the `eval()` function.
3. The code should represent a solution to the query.
4. PRINT ONLY THE EXPRESSION.
5. Do not quote the expression.
"""

You can update it as well:

In [ ]:
new_prompt = PromptTemplate("""\
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}
Query: {query_str}

Expression: """)

query_engine.update_prompts({"pandas_prompt": new_prompt})

### Implementing Query Engine using Query Pipeline Syntax

Here we show you how to construct the Pandas Query Engine using our Query Pipeline syntax and the prompt components above. This gives you a greater overview of the underlying components.

TODO: move this to separate notebook under pipelines.

In [18]:
from llama_index.query_pipeline import QueryPipeline as QP, Link
from llama_index.query_engine.pandas import PandasInstructionParser
from llama_index.llms import OpenAI

In [ ]:
pandas_prompt = prompts["pandas_prompt"].partial(
    instruction_str=instruction_str,
    df_str=df.head(5)
)
response_synthesis_prompt = prompts["response_synthesis_prompt"].partial(
    
)
pandas_output_parser = PandasInstructionParser(df)


qp = QueryPipeline(modules={
    "pandas_prompt": pandas_prompt,
    "llm1": llm,
    "pandas_output_parser": pandas_output_parser,
    "response_synthesis_prompt": response_synthesis_prompt,    
})
qp.add_chain(["pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    Link("llm1", "response_synthesis_prompt", dest_key="pandas_instruction"),
    Link("pandas_output_parser", "response_synthesis_prompt", dest_key="pandas_output")
)


In [ ]:
response = qp.run(query_str="What is the correlation between survival and age?")
print(str(response))